In [1]:
import requests
import pandas as pd
import time

In [2]:
def get_data_for_page(url, page):
    # Parâmetros da solicitação GET
    params = {'page': page, 'pageSize': 50}
    
    # Fazendo a solicitação GET
    response = requests.get(url, params=params)
    
    # Verificando se a solicitação foi bem sucedida
    if response.status_code == 200:
        # Convertendo a resposta para JSON e retornando os itens
        data = response.json()
        return data.get('items', [])
    else:
        # Se não for bem sucedida, exibimos uma mensagem de erro
        print(f"Erro na página {page}: {response.status_code}")
        return None

In [3]:
# URL do endpoint
url = 'https://api.fbi.gov/wanted/v1/list'

In [4]:
# Lista para armazenar os dados de todas as páginas
data_list = []

In [5]:
# Número total de páginas
total_pages = 20

In [6]:
# Tamanho do lote de páginas a serem processadas de uma vez
batch_size = 5

In [7]:
# Tempo de espera entre os lotes de solicitações (em segundos)
wait_time = 40

In [8]:
# Fator de aumento do tempo de espera entre tentativas
wait_time_multiplier = 1.5

In [9]:
# Loop através das páginas em lotes
for batch_start in range(1, total_pages + 1, batch_size):
    # Obter páginas neste lote
    batch_pages = range(batch_start, min(batch_start + batch_size, total_pages + 1))
    
    # Loop através das páginas neste lote
    for page in batch_pages:
        # Obter dados para esta página
        page_data = get_data_for_page(url, page)
        
        # Se os dados foram recebidos com sucesso, adicione-os à lista de dados
        if page_data is not None:
            data_list.extend(page_data)
    
    # Aguardar um intervalo de tempo após o processamento de cada lote
    time.sleep(wait_time)

Erro na página 14: 429
Erro na página 15: 429
Erro na página 16: 429
Erro na página 17: 429
Erro na página 18: 429
Erro na página 19: 429
Erro na página 20: 429


In [10]:
# Convertendo a lista de dicionários para um DataFrame do pandas
df = pd.DataFrame(data_list)

In [11]:
# Selecionando apenas as colunas desejadas
df = df[['title', 'sex', 'eyes_raw', 'nationality', 'hair', 'weight', 'race', 'dates_of_birth_used', 'subjects']]

In [12]:
# Renomeando as colunas
df = df.rename(columns={'title': 'nome', 'sex': 'sexo', 'eyes_raw': 'cor_olhos', 
                        'nationality': 'nacionalidade', 'hair': 'cor_cabelo', 
                        'weight': 'peso', 'race': 'raca', 
                        'dates_of_birth_used': 'data_nascimento', 'subjects': 'delitos'})

In [13]:
# Exibindo o DataFrame resultante
print(df)

                                            nome    sexo   cor_olhos  \
0        MAURA MURRAY - HAVERHILL, NEW HAMPSHIRE  Female  Green/Blue   
1                          ROBERT WILLIAM FISHER    Male        Blue   
2    DAWN MARIE MOZINO - BRYN MAWR, PENNSYLVANIA  Female       Brown   
3                 JANE DOE - LINN COUNTY, OREGON  Female        None   
4            OREGON RIGHT TO LIFE BUILDING ARSON    None        None   
..                                           ...     ...         ...   
645                            DAWNITA WILKERSON  Female       Brown   
646                       OLIVIA SAMANTHA FOWLER  Female       Green   
647               JOHN DOE - WICKENBURG, ARIZONA    Male        None   
648                                 NAKHLE NADER    Male       Brown   
649                             AVERY WHITESKUNK    Male       Brown   

    nacionalidade cor_cabelo                                           peso  \
0            None      brown                            

In [14]:
print(df[df['nome'] == 'AARON PAUL VICTORY'])

                 nome  sexo cor_olhos nacionalidade cor_cabelo  \
8  AARON PAUL VICTORY  Male     Brown          None      brown   

                peso   raca  \
8  196 to 201 pounds  white   

                                     data_nascimento  \
8  [January 28, 1979, July 28, 1979, January 28, ...   

                                         delitos  
8  [Additional Violent Crimes, Case of the Week]  


In [16]:
# Salvando o DataFrame alterado em um arquivo Excel
df.to_excel('df.xlsx', index=False)

In [17]:
# Separar os delitos em linhas individuais
new_data = []
for index, row in df.iterrows():
    for delito in row['delitos']:
        new_row = row.copy()
        new_row['delito'] = delito
        new_data.append(new_row)

In [18]:
# Criar o DataFrame alterado
df_alterado = pd.DataFrame(new_data)

In [19]:
# Exibindo o DataFrame resultante
print(df_alterado)

                                            nome    sexo   cor_olhos  \
0        MAURA MURRAY - HAVERHILL, NEW HAMPSHIRE  Female  Green/Blue   
1                          ROBERT WILLIAM FISHER    Male        Blue   
2    DAWN MARIE MOZINO - BRYN MAWR, PENNSYLVANIA  Female       Brown   
3                 JANE DOE - LINN COUNTY, OREGON  Female        None   
4            OREGON RIGHT TO LIFE BUILDING ARSON    None        None   
..                                           ...     ...         ...   
646                       OLIVIA SAMANTHA FOWLER  Female       Green   
647               JOHN DOE - WICKENBURG, ARIZONA    Male        None   
648                                 NAKHLE NADER    Male       Brown   
649                             AVERY WHITESKUNK    Male       Brown   
649                             AVERY WHITESKUNK    Male       Brown   

    nacionalidade cor_cabelo               peso      raca   data_nascimento  \
0            None      brown  120 to 125 pounds     whit

In [24]:
for index, row in df_alterado.iterrows():
    if isinstance(row['data_nascimento'], list):  # Verificar se é uma lista
        df_alterado.at[index, 'data_nascimento'] = ', '.join(row['data_nascimento'])  # Juntar os elementos da lista em uma string
    else:
        print(row['data_nascimento']) # Se não for uma lista, imprimir o valor diretamente

None
None
None
None
January 28, 1979, July 28, 1979, January 28, 1969
None
None
None
None
October 31, 1963
May 31, 1974, August 26, 1972
None
None
October 16, 1966
1963
Approximately 1960
Unknown
July 10, 1965, July 11, 1965
June 13, 1964
June 26, 1967
February 10, 1967
January 30, 1966
None
None
None
October 12, 1979
None
None
None
None
None
None
November 26, 2003
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
October 11, 1985
None
None
None
None
None
None
None
None
None
March 9, 1990
None
None
None
None
None
None
None
None
December 4, 1978
None
None
None
None
None
None
None
None
None
None
None
None
None
August 14, 1974
None
None
None
None
None
None
None
None
August 31, 1981
None
October 4, 1990
October 4, 1990
October 23, 1979
None
None
None
None
None
May 26, 1970
None
None
None
None
None
None
None
November 7, 1979
None
None
May 5, 1997
May 5, 1997
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None

In [25]:
# Remover a coluna 'delitos'
df_alterado.drop(columns=['delitos'], inplace=True)

In [26]:
# Exibir o DataFrame resultante
print(df_alterado)

                                            nome    sexo   cor_olhos  \
0        MAURA MURRAY - HAVERHILL, NEW HAMPSHIRE  Female  Green/Blue   
1                          ROBERT WILLIAM FISHER    Male        Blue   
2    DAWN MARIE MOZINO - BRYN MAWR, PENNSYLVANIA  Female       Brown   
3                 JANE DOE - LINN COUNTY, OREGON  Female        None   
4            OREGON RIGHT TO LIFE BUILDING ARSON    None        None   
..                                           ...     ...         ...   
646                       OLIVIA SAMANTHA FOWLER  Female       Green   
647               JOHN DOE - WICKENBURG, ARIZONA    Male        None   
648                                 NAKHLE NADER    Male       Brown   
649                             AVERY WHITESKUNK    Male       Brown   
649                             AVERY WHITESKUNK    Male       Brown   

    nacionalidade cor_cabelo               peso      raca data_nascimento  \
0            None      brown  120 to 125 pounds     white 

In [27]:
print(df_alterado[df_alterado['nome'] == 'AARON PAUL VICTORY'])

                 nome  sexo cor_olhos nacionalidade cor_cabelo  \
8  AARON PAUL VICTORY  Male     Brown          None      brown   
8  AARON PAUL VICTORY  Male     Brown          None      brown   

                peso   raca  \
8  196 to 201 pounds  white   
8  196 to 201 pounds  white   

                                     data_nascimento  \
8  January 28, 1979, July 28, 1979, January 28, 1969   
8  January 28, 1979, July 28, 1979, January 28, 1969   

                      delito  
8  Additional Violent Crimes  
8           Case of the Week  


In [28]:
# Salvando o DataFrame alterado em um arquivo Excel
df_alterado.to_excel('df_alterado.xlsx', index=False)